In [ ]:
# ----------------------------------------------------------------------------------------------------------------
# Problem Formulation Overview
# ----------------------------------------------------------------------------------------------------------------

"""
We want to find the optimal trading strategy that maximizes expected log wealth over a given horizon.
We start with a simple formulation in terms of wealth and contracts held (v1).
This quickly becomes infeasible as the horizon increases or the contract price becomes more volatile.
We then introduce coverage ratio and coverage ratio change (v2), which greatly reduces the size of the DP table.
However, coverage ratio can still explode for extreme cases, so we introduce coverage proportion (v3).
Finally, we add market spreads to the model (v4).
"""

# ----------------------------------------------------------------------------------------------------------------
# "Why v1 < v2 < v3 < v4"
# ----------------------------------------------------------------------------------------------------------------

"""
Let T = |time steps|, W = |wealth steps|, x = |contract steps|, c = |contract price steps|, R = |Price transitions| (c_t | c_{t-1})
Let z = |coverage ratio steps|, a = |coverage ratio change steps|
Let theta = |coverage proportion start steps|, b = |coverage proportion end steps| (size of theta = size of b)

In v1: 
  Memory complexity: O(T * W * c)
  Time complexity: O(T * W * x * c * R) 
  This is because for each sell of the DP table we compute over all possible actions (x) and price transitions (R)
  Note that c and R are bounded between 0 and 1, and T is finite, but W and x can be arbitrarily large, so this problem formulation is infeasible

In v2 we noticed that optimal action and values are independent of W and x, so we introduce coverage ratio z and coverage ratio change a:
  Memory complexity: O(T * z * c)
  Time complexity: O(T * z * a * c * R)
  This removes the dependency on W and x,
  but z & a can explode in size for edge cases of large W and small xs'; this problem formulation is still infeasible

In v3 we introduce coverage proportion at start and end of each time step as state & action variables theta & b respectively:
  Memory complexity: O(T * theta * c)
  Time complexity: O(T * theta * b * c * R)
  Since theta, b, c, R are all bounded between 0 and 1, with sensible grid sizes, the DP table for time step t is O(1) memory and time. 

In v4 we add market spreads to the model as a percentage change on the mid price:
  Memory complexity: O(T * theta * c)
  Time complexity: O(T * theta * b * c * R)
  The spreads are defined as a percentage of the mid price, so theta is unchanged. 
  The immediate reward calculation is different, but the DP table for time step t is still O(1) memory and time. 
"""

# ----------------------------------------------------------------------------------------------------------------
# Insights & Application:
# ----------------------------------------------------------------------------------------------------------------

"""
Computational Feasibility: The final version is feasible to run on a personal computer for long duration trades or granular time frames. 
Position Allocation: Furthermore since action and state are relative to total position value at time t, DP grid doesn't blow up when position allocation -> 0 or 1.  
Lookup Table: DP table is lookup table, so no runtime computation is needed. 
Model Inputs: We only need to provide the subjective probability of the outcome, market spreads, and contract price transition matrix.
Providing true price transition matrix may be implausible, but we hypothesize that model will perform well even with basic price transition matrix assumptions.
In other words we think that model value comes from baking in price volatility & market frictions rather than true price dynamics.
"""

# ----------------------------------------------------------------------------------------------------------------
# Story & Bellman Equation: 
# ----------------------------------------------------------------------------------------------------------------

"""
Our final problem formulation (v4) has the following story. 
  State variable: at the start of each time step t our position value is made up of cash wealth (W_{t-1}) and contract value (x_{t-1} * c_t).
  Action variable: at the end of each time step t our position value is made up of cash wealth (W_t) and contract value (x_t * c_t).
  Our position value didn't change because contract price only updates between time steps, but our action is the allocation of wealth between cash and contract.

This is concretely broken down into 3 components in our Bellman Equation:
  immediate_cost from action: log(1 - theta_t) ~> prop. of position value in cash wealth at start of time step
  immediate_reward from action: log(1 - b_t) or piecewise w/ market spreads ~> cash wealth gained/lost from selling/buying contracts
  expectation of future reward from action: v_{t+1}(theta_{t+1}, c_{t+1}) ~> cash wealth gained/lost from optimal play given current action
"""

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
# v1 / Problem statement: Derive in terms of wealth and contracts held
# ----------------------------------------------------------------------------------------------------------------

"""
Let contract price = c_t, number of contracts held = x_t, wealth = w_t, change of contracts held = u_t
T-1 trading time steps, T-th time step is terminal time step (c_T resolves to 1 or 0)
Assume no discounting for Bellman Equation (short duration contracts)

With initial conditions at time 0: c_0, x_t = 0, w_t = w_0, u_t = 0

At each time 1 < t < T:
Contract change: x_t = x_{t-1} + u_t
Wealth change: W_t = W_{t-1} - c_t * u_t

And terminal value condition t=T: V_T = plog(W_T + x_T * c_T) + (1-p)log(W_T), where p is subjective probability of outcome
Seek to maximize expected log wealth V_t = E[log(W_t)] = log(W_{t-1}) + v_t(x_t, c_t)

Subsequent versions iterate through different state & action variables, and calculate their effect on wealth for the Bellman Equation. 
"""

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
# v2: Derive in terms of coverage ratio and coverage ratio change
# ----------------------------------------------------------------------------------------------------------------

"""
The issue with v1 is the size of DP table depends on W, which can be arbitrarily large as time steps increase
However, the optimal shares to trade at time t doesn't depend on how much wealth you have, but rather the proportion of current wealth you should put into shares / take out as cash
Therefore, we hope to turn (w_t, u_t) into (z_t, a_t), where z_t is the coverage ratio of wealth to shares and a_t is the change in coverage ratio

More formally at each time t:
Coverage Ratio: 
  z_t = x_{t-1} * c_t / W_{t-1}
Coverage Ratio change: 
  a_t = u_t * c_t / W_{t-1}
And price change: 
  R_t = c_{t+1} / c_t
Since -x_{t-1} ≤ u_t ≤ W_{t-1}/c_t => -z_t ≤ a_t ≤ 1

In our new problem, we have initial conditions:
  c_0, z_0 = 0, a_0 = 0 
And terminal value condition: 
  v_T = plog(1 + z_T / c_{T-1}) # coverage ratio is z_T * R_t at time t, with price transitioning from c_{T-1} to 1 w.p. p

Then for all 1 ≤ t < T:
W_t = W_{t-1} - u_t * c_t = W_{t-1} (1 - a_t)
x_t = x_{t-1} + a_t/c_t * W_{t-1}
z_{t+1} = x_t * c_{t+1} / W_t
        = (c_{t+1} / c_t) * (x_t * c_t / W_t)
        = (c_{t+1} / c_t) * ((x_{t-1} + a_t/c_t * W_{t-1}) * c_t / (W_{t-1} (1 - a_t)))
z_{t+1} = (c_{t+1} / c_t) * (z_t + a_t) / (1 - a_t) 

Let V_T be the expected log wealth, which we are trying to maximize: V_T = E[log(W_T)]
Any V_t, 1 ≤ t ≤ T, must satisfy: V_t = log(W_t) + v_t(z_t, c_t), where v_t is the expected wealth multiplier from coverage ratio z_t and contract price c_t
Similarly, 
  V_{t+1} = log(W_{t+1}) + v_{t+1}(z_{t+1}, c_{t+1}) 
          = log(W_t * (1 - a_t)) + v_{t+1}(z_{t+1}, c_{t+1})
          = log(W_t) + log(1 - a_t) + v_{t+1}(z_{t+1}, c_{t+1})
Hence v_t = max_{a_t} E[log(1 - a_t) + v_{t+1}(z_{t+1}, c_{t+1})], ~> expectation over future price change
where -z_t ≤ a_t ≤ 1
"""

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
# v3: Derive in terms of contract / (contract + wealth) allocation
# ----------------------------------------------------------------------------------------------------------------

"""
The issue with v2 is that coverage ratio can still blow up when W is large or contract price is small, so the DP table still can be arbitrarily large
Instead for our state variable we use coverage proportion at the start of the time step which is always in [0, 1].
Our action variable becomes the coverage proportion at the end of the time step, which is also in [0, 1].
This lets us reduce a potentially infinite (state, action) space to a 

Crucially, the idea is total postion value doesn't change at the same time step:
  Let P_t be the total position (cash wealth + contract) worth at time t, then:
      P_t = W_{t-1} + x_{t-1} * c_t = W_t + x_t * c_t

  Prop. fraction in contracts at start of time t: 
      theta_t = x_{t-1} * c_t / (W_{t-1} + x_{t-1} * c_t)
  Prop. fraction in contracts at end of time t: 
      b_t = x_t * c_t / (W_t + x_t * c_t) , 0 ≤ b_t ≤ 1
  Price Change: 
      R_t = c_{t+1} / c_t

Initial conditions t=0:
  theta_0 = 0, b_0 = 0, c_0
And terminal condition t=T:
  V_T = plog(W_T + x_T * c_T) + (1-p)log(W_T)
  V_T = plog(W_T + theta_T / (1-theta_T) * W_T / c_{T-1}) + (1-p)log(W_T) # W * coverage ratio * mark to market 1/c_{T-1}
  V_T = log(W_T) + p * log[1 + theta_T / (c_{T-1} * (1-theta_T))]
  Rewritten in recursive form:
      v_T(theta_T, c_{T-1}) = p * log[1 + theta_T / (c_{T-1} * (1-theta_T))]

For all 1 ≤ t < T:
theta_{t+1} = x_t * c_{t+1} / (W_t + x_t * c_{t+1}) = b_t * R_t / [(1-b_t) + b_t * R_t]
W_t = (1-b_t) * P_t
W_t/W_{t-1} = (1-b_t) * P_t / W_{t-1} = (1-b_t) * (W_{t-1} + x_{t-1} * c_t) / W_{t-1} = (1-b_t) / (1 - theta_t)
So log W_t = log W_{t-1} + log(1-b_t) - log(1-theta_t)
And v_t(theta_t, c_t) = -log(1-theta_t) + max_{b_t} E[log(1-b_t) + v_{t+1}(theta_{t+1}, c_{t+1})], b in [0, 1]
"""

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
# v4: Add spreads
# ----------------------------------------------------------------------------------------------------------------

"""
Finally, we add market spreads to simulate real trading conditions.
Ideally model runs in high liquidity & volatility environments and trades over a 10-30min window,
but we will assume worst case: 
  c_t = mid price at time t
  buy price cb_t = c_t * (1 + gamma_b), gamma_b ~ 5%
  sell price cs_t = c_t * (1 - gamma_s), gamma_s ~ 10%

The state variable theta_t is defined using mid prices, so unchanged from v3:
  theta_t = x_{t-1} * c_t / (W_{t-1} + x_{t-1} * c_t)

The action variable b_t is also defined using mid prices (what we target):
  b_t = x_t * c_t / (W_t + x_t * c_t) , 0 ≤ b_t ≤ 1  (using mid prices)

The change is in the WEALTH EVOLUTION, not in the definition of b_t.

From v3 (no spreads): immediate_reward = log(1 - b_t)
With spreads, immediate_reward becomes:

If b_t > theta_t (buying contracts):
  immediate_reward = log[1 - theta_t - (b_t - theta_t)(1 + gamma_b)]
If b_t < theta_t (selling contracts):
  immediate_reward = log[1 - theta_t - (b_t - theta_t)(1 - gamma_s)]
If b_t = theta_t (no trade):
  immediate_reward = log(1 - b_t) = log(1 - theta_t)

Bellman equation (otherwise unchanged from v3):
  v_t(theta_t, c_t) = -log(1 - theta_t) + max_{b_t} E_t[ immediate_reward + v_{t+1}(theta_{t+1}, c_{t+1})]
where theta_{t+1} = (R_t * b_t) / (1 - b_t + R_t * b_t), R_t = c_{t+1}/c_t

Terminal condition unchanged:
  v_T(theta_T, c_{T-1}) = p * log[1 + theta_T / (c_{T-1} * (1 - theta_T))]

Only difference from v3 is piecewise immediate_reward calculation.
"""